# Neuroscience using `fastplotlib` and `pynapple`

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with `pynapple`, to show how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
import fastplotlib as fpl
import pynapple as nap
import numpy as np
from ipywidgets import IntSlider, Layout, VBox, HBox

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

MESA-INTEL: warning: cannot initialize blitter engine
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


Available devices:
🯄 (default) | Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | Vulkan | Mesa 24.0.8-1
❗ | llvmpipe (LLVM 17.0.6, 256 bits) | CPU | Vulkan | Mesa 24.0.8-1 (LLVM 17.0.6)
❗ | Mesa Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | OpenGL | 


In [3]:
import warnings
warnings.simplefilter('ignore')

## Load the data 

#### TODO: INSERT SUMMARY OF THE DATA
#### TODO: ADD INSTRUCTIONS FOR CHANGING PATHS TO WHEREVER DATASET IS STORED IN BINDER

In [4]:
data = nap.load_file("./data.nwb")

In [5]:
data

data
┍━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                  │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ position_time_support │ IntervalSet │
│ RoiResponseSeries     │ TsdFrame    │
│ calcium_video         │ TsdTensor   │
│ beh_video             │ TsdTensor   │
│ z                     │ Tsd         │
│ y                     │ Tsd         │
│ x                     │ Tsd         │
│ rz                    │ Tsd         │
│ ry                    │ Tsd         │
│ rx                    │ Tsd         │
┕━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙

### Let's view the behavior and calcium data

Hopefully, by the end of the summer we will have developed a tool ([`pynaviz`](https://github.com/pynapple-org/pynaviz)) that makes these visualizations and synchronizations even easier :D

In [6]:
# behavior shape
behavior_data = data["beh_video"]
behavior_data.shape

(9045, 204, 256)

In [7]:
# calcium shape
calcium_data = data["calcium_video"]
calcium_data.shape

(17886, 136, 166)

In [8]:
behavior_fr = 7.5
calcium_fr = 15

#### We are going to minimize our view of the data to where both behavior and position data are available:

In [9]:
frame_min = data["position_time_support"]["start"][0]
frame_max = data["position_time_support"]["end"][0]
(frame_min, frame_max)

(7.39305, 1213.22765)

#### Create a plot for calcium and behavior video

In [10]:
nap_figure = fpl.Figure(shape=(1,2), names=[["raw", "behavior"]])

nap_figure["raw"].add_image(data=calcium_data[0], name="raw_frame")
nap_figure["behavior"].add_image(data=behavior_data[0], cmap="gray")

RFBOutputContext()

Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


<weakproxy at 0x7f0057be08b0 to ImageGraphic at 0x7f004b30f910>

#### Create a slider that updates behavior and calcium videos so they are aligned

In [11]:
# This time will be in milliseconds
synced_time = IntSlider(min=frame_min, max=frame_max, description="s", layout=Layout(width="20%"))

def update_time(change):
    # get the index of synced slider
    time_s = change["new"]
    # get the corresponding calcium frame from the pynapple tensor
    frame_raw = calcium_data.get(time_s, time_units="s")
    # update the data in the plot
    nap_figure["raw"].graphics[0].data = frame_raw
    # get the corresponding behavior frame from the pynapple tensor
    frame_behavior = behavior_data.get(time_s, time_units="s")
    # update the data in the plot
    nap_figure["behavior"].graphics[0].data = frame_behavior
    
synced_time.observe(update_time, "value")

In [12]:
VBox([nap_figure.show(), synced_time])

## Make a plot of the calcium traces 

In [13]:
# create a figure
temporal_fig = fpl.Figure()

RFBOutputContext()

In [14]:
# get the temporal data from the nwb notebook
temporal = data["RoiResponseSeries"]
temporal

Time (s)           0        1        2        3        4  ...
-----------  -------  -------  -------  -------  -------  -----
0.0          0        0.43582  2.96331  0        0        ...
0.033333     0        0.43406  2.95294  0        0        ...
0.066667     0        0.43231  2.9426   0        0        ...
0.1          0        0.43057  2.93231  0        0        ...
0.133333     0        0.42883  2.92205  0        0        ...
0.166667     0        0.4271   2.91182  0        0        ...
0.2          0        0.42537  2.90163  0        0        ...
...
1192.166667  2.54202  0.14531  0.44013  0.5681   0.65477  ...
1192.2       2.53029  0.14775  0.43842  0.56657  0.65227  ...
1192.233333  2.51861  0.14962  0.43671  0.56505  0.64979  ...
1192.266667  2.50698  0.15104  0.435    0.56354  0.64731  ...
1192.3       2.49541  0.15209  0.43331  0.56202  0.64485  ...
1192.333333  2.48389  0.15283  0.43162  0.58476  0.64239  ...
1192.366667  2.47242  0.15333  0.42994  0.62802  0.63994  ...
dt

In [15]:
# we need to transpose our temporal data so that it is (# components, time (s))
temporal_graphic = temporal_fig[0,0].add_line_stack(data=temporal.to_numpy().T, colors="w")

#### We can also add a `LinearSelector` that we can then map to our behavior and calcium videos

In [16]:
temporal_slider = temporal_graphic.add_linear_selector()

In [17]:
temporal_fig.show(maintain_aspect=False)

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

#### We can another function to update the position of the `LinearSelector` based on the position of our `IntSlider` from before

In [18]:
def update_time_temporal(change):
    # get the index of synced slider
    time_s = change["new"]

    # temporal data has framerate 30
    temporal_slider.selection = time_s * 30

synced_time.observe(update_time_temporal, "value")

#### Let's view everything together

In [20]:
VBox([nap_figure.show(), temporal_fig.show(maintain_aspect=False), synced_time])

#### Let's add an event handler that allows us to select a given component

In [64]:
@temporal_graphic.add_event_handler("click")
def click_event(ev):
    # reset the component colors
    temporal_graphic.colors = "w"
    # set the selected component color
    ev.graphic.colors = "magenta"

#### Now that we are selecting a given components, let's also create a plot that will allow us to zoom in on that given neural trace

In [73]:
zoomed_fix = fpl.Figure()

RFBOutputContext()

In [74]:
@temporal_graphic.add_event_handler("click")
def update_zoom_plot(ev):
    zoomed_fix[0,0].clear()
    lg = zoomed_fix[0,0].add_graphic(ev.graphic)

In [75]:
zoomed_fix.show(maintain_aspect=False)

/home/caitlin/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/caitlin/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/caitlin/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [ ]:
# add contours
# single plot with contours

In [5]:
contours = data.nwb.processing['ophys']['ImageSegmentation']['PlaneSegmentation']['image_mask'].data[:]
contours = contours.reshape((len(contours), 166, 136))

In [6]:
contours.shape

(105, 166, 136)

In [7]:
len(list(contours))

105

In [9]:
list(contours)[0].shape

(166, 136)

In [11]:
contours.sum(0).shape

(166, 136)

In [13]:
fig = fpl.Figure()

fig[0,0].add_image(contours[0])

fig.show()

RFBOutputContext()

/home/caitlin/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/caitlin/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/caitlin/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…